# 1. Obtaining data for 'Teams' 

## - create directory to store downloaded files, obtain data for each time, and save to a csv file

In [1]:
import requests
import os

year = 2019
base_url = "https://www.basketball-reference.com"
url = f"https://www.basketball-reference.com/leagues/NBA_{year}.html"

data = requests.get(url)

# create directory
dir_name = 'data'
try:
    os.makedirs(dir_name)
except OSError as error:
    print(f'Directory "{dir_name}" cannot be created')

# download data and save into html file for data parsing, instead of downloading the entire page each time
with open(f"data/{year}.html", "w+", encoding="cp437", errors='ignore') as f:
    f.write(data.text)


In [2]:
from bs4 import BeautifulSoup

with open(f"data/{year}.html") as f:
    # read the file and store the data as a string obj
    page = f.read()

# initialize soup object to parse html data
soup = BeautifulSoup(page, "html.parser")

# get the abbreviated team names and store into a list (some abbreviated names are different on this site)
nba_team_pages = []

# find the appropriate tables with the team stats for every team in the NBA
eastern_conf_table = soup.find(id="confs_standings_E")
western_conf_table = soup.find(id="confs_standings_W")

# eastern conference teams
for a in eastern_conf_table.find_all('a'):
    nba_team_pages.append(a['href'])

# western conference teams
for a in western_conf_table.find_all('a'):
    nba_team_pages.append(a['href'])

# team abbrs
team_abbrs = []

for abbr in nba_team_pages:
    team_abbrs.append(abbr.split('/')[2])

# at this step, you will have a list of abbreviated team names that matches the format the site uses


In [3]:
# from selenium import webdriver
# import time

# create directory
parent_dir = dir_name
dir_name = os.path.join(parent_dir, 'teams')

try:
    os.makedirs(dir_name)
except OSError as error:
    print(f'Directory "{dir_name}" cannot be created')


In [4]:
# go through each page and save them to their corresponding html files
for abbr_name in team_abbrs:
    # use Selenium here to allow page content that require Javascript to be rendered

    url = f"https://www.basketball-reference.com/teams/{abbr_name}/{year}.html"

    html_data = requests.get(url)

    # download data and save into html file for data parsing, instead of downloading the entire page each time
    with open(f"{dir_name}/{abbr_name}.html", "w+", encoding="cp437", errors='ignore') as f:
        f.write(html_data.text)

# at this step, you will have the html files of all the individual teams


In [5]:
# sort the names to match directory order
team_abbrs.sort()


In [6]:
import pandas as pd
import re

teams_df = pd.DataFrame(
    columns=['Name', 'Abbreviated Name', 'Arena', 'Wins', 'Losses', 'W-L'])

for idx, team in enumerate(team_abbrs, start=1):
    # each team should have a name, abbr name, wins, losses, location
    with open(f'data/teams/{team}.html', encoding="cp437", errors='ignore') as f:
        page = f.read()

        # initialize soup object to parse html data
        soup = BeautifulSoup(page, "html.parser")

        # team name
        team_name = soup.find(id="info").find_all('span')[1].extract().text

        # wins/losses
        wins_and_losses = soup.find(id="info").select_one('div[data-template="Partials/Teams/Summary"]').find(
            'p').extract().text.replace(" ", "").replace("\n", "").split(",")[0].split(":")[1]
        wins = wins_and_losses.split("-")[0]
        losses = wins_and_losses.split("-")[1]

        # location i.e. arena
        # filter string to find the arena name
        locations = soup.find(id="info").select_one(
            'div[data-template="Partials/Teams/Summary"]').find_all('p')
        pattern = 'Attendance'
        location = None
        match = None

        # find the tag with the arena
        for loc in locations:
            match = (re.search(pattern, loc.text))
            if match:
                location = loc.text
                break

        # # obtain only the arena name, and filter further
        # location = location[:match.start()].replace("\n", "").replace(" ", "").split(":")[-1]
        location = location[:match.start()].replace(
            "\n", "").split(":")[-1].strip()

        teams_df.loc[idx] = [team_name, team,
                             location, wins, losses, wins_and_losses]


In [72]:
team_names = list(teams_df.loc[:, 'Name'])

In [287]:
# convert the wins and lossees columns from strings to ints
teams_df['Wins'] = pd.to_numeric(teams_df['Wins'])
teams_df['Losses'] = pd.to_numeric(teams_df['Losses'])

In [288]:
# create directory
dir_name = 'csv_files'

try:
    os.makedirs(dir_name)
except OSError as error:
    print(f'Directory "{dir_name}" cannot be created')

teams_df.to_csv('csv_files/teams.csv')


Directory "csv_files" cannot be created


## 2. Obtain team stats data for each team, filter data, convert to dataframe, and save to csv file

In [225]:
from bs4 import Comment

teams_stats_df = pd.DataFrame(
    columns=[
        # "Name",
        "Field Goals",
        "Field Goals Attempted",
        "Field Goal %",
        "3-Point Field Goals",
        "3-Point Field Goals Attempted",
        "3-Point Field Goal %",
        "2-Point Field Goals",
        "2-Point Field Goals Attempted",
        "2-Point Field Goal %",
        "Free Throws",
        "Free Throws Attempted",
        "Free Throw %",
        "Offensive Rebounds",
        "Defensive Rebounds",
        "Total Rebounds",
        "Assists",
        "Steals",
        "Blocks",
        "Turnovers",
        "Personal Fouls",
        "Points",
    ]
)


for idx, team in enumerate(team_abbrs, start=1):
    # each team should have a name, abbr name, wins, losses, location
    with open(f'data/teams/{team}.html', encoding="cp437", errors='ignore') as f:
        page = f.read()

        # initialize soup object to parse html data
        soup = BeautifulSoup(page, "html.parser")

        comments = soup.find_all(string=lambda text: isinstance(text, Comment))

        # filter for the commented tables, and returns a list of tables and their contents
        table_comments = list(
            filter(lambda comment: 'table' in str(comment), comments))

        # obtain the team stats table
        team_table = []
        for comment in table_comments:
            try:
                team_table.append(pd.read_html(
                    comment, attrs={'id': 'team_and_opponent'})[0])
            except:
                continue

        team_df = team_table[0]
        
        # filter the data, replace 'nan' with 0 values
        team_df = team_df.fillna(0)

        # filter for only the first columns where the target data is
        team_stats_dict = dict(team_df.iloc[1])

        # removing columns that don't need
        team_stats_list = [
            stat
            for (category, stat) in team_stats_dict.items()
            if category not in ["Unnamed: 0", "G", "MP"]
        ]
        
        # add to dataframe
        teams_stats_df.loc[idx] = (team_stats_list)

# append the team names as a column in the table
teams_stats_df['Name'] = team_names



Directory "csv_files" cannot be created


In [295]:
# need to adjust column data types from objects to floats

# iterate through the column names, and change all column types to float64
for col in list(teams_stats_df.keys()):
    if col != 'Name':
        teams_stats_df = teams_stats_df.astype({col: float})

In [297]:
# create directory
dir_name = 'csv_files'

try:
    os.makedirs(dir_name)
except OSError as error:
    print(f'Directory "{dir_name}" cannot be created')

teams_stats_df.to_csv('csv_files/teams_stats.csv')

Directory "csv_files" cannot be created


# 3. Obtain player basic info i.e. Name, Date of Birth, Position, Team and save to csv file

In [226]:
from datetime import datetime

In [180]:
players_df = pd.DataFrame(
    columns=[
        'Name',
        'Date of Birth',
        'Position',
        'Team'
    ]
)

# for each team, it contains data of their roster and their stats
# each player name contains a link to their profile page with the necessary data
for team in team_abbrs:
    with open(f'data/teams/{team}.html', encoding="cp437", errors='ignore') as f:
        page = f.read()

        # initialize soup object to parse html data
        soup = BeautifulSoup(page, "html.parser")
        
        # for each time, find a list of the players and their <a> tag pointing to their profile
        players_stats_list = soup.find('table', id="per_game").find_all('a')
        
        # filter the list
        players_stats_links = [ a for a in players_stats_list if '/gamelog/' not in str(a) ]
        
        # obtain only the links, not the entire <a> tag
        players_stats_links = [ str(href).split('"')[1] for href in players_stats_links ]
        
        # obtain the team name
        team_name = soup.find(id="info").find_all('span')[1].extract().text
        
        for url in players_stats_links:
            # go to url of specific player
            html_data = requests.get(base_url + url)
            
            # get the html content of the url
            html = html_data.text
            
            # need to extract Name, Age, Position, Team
            # initialize soup object to parse html data
            soup = BeautifulSoup(html, "html.parser")
            info = soup.find(id='meta').find('div', class_=lambda class_name: class_name != 'media-item')
            
            # name
            name = info.find('h1').find('span').text
            
            # date of birth
            dob = info.find('span', id=lambda _id: _id == 'necro-birth').text.replace('\n', "").split(',')
            dob = " ".join([element.strip() for element in dob])
            dob_obj = datetime.strptime(dob, "%B %d %Y")
            dob = str(dob_obj).split(' ')[0]
            
            # position
            position_element = [p for p in info.find_all('p') if 'Position' in str(p)]
            position = position_element[0].text.replace("\n", "").strip().replace("▪", "").replace("         ", ",").split(", ")[0].split(":")[1].strip()    
            
            # add it as a new row into players_df at the end
            players_df.loc[players_df.shape[0]] = [name, dob, position, team_name]


# players_df

,Name,Date of Birth,Position,Team
0,Trae Young,1998-09-19,Point Guard,Atlanta Hawks
1,John Collins,1997-09-23,Power Forward,Atlanta Hawks
2,Taurean Prince,1994-03-22,Power Forward and Small Forward,Atlanta Hawks
3,Kevin Huerter,1998-08-27,Shooting Guard,Atlanta Hawks
4,Dewayne Dedmon,1989-08-12,Center,Atlanta Hawks
...,...,...,...,...
617,Ron Baker,1993-03-30,Shooting Guard,Washington Wizards
618,Jason Smith,1986-03-02,Center and Power Forward,Washington Wizards
619,Gary Payton II,1992-12-01,Point Guard and Small Forward,Washington Wizards
620,John Jenkins,1991-03-06,Shooting Guard,Washington Wizards


In [298]:
# change type of dob to datetime64
players_df['Date of Birth'] = pd.to_datetime(players_df['Date of Birth'])

In [299]:
# adjust indexing on current data frame
players_df.index += 1

# save it to a csv file

# create directory
dir_name = 'csv_files'

try:
    os.makedirs(dir_name)
except OSError as error:
    print(f'Directory "{dir_name}" cannot be created')

players_df.to_csv('csv_files/players.csv')

Directory "csv_files" cannot be created


# 4. Obtain player stats and save to csv file

In [282]:
players_stats_df = pd.DataFrame(
    columns=[
        "Name",
        "Minutes Played",
        "Field Goals",
        "Field Goals Attempted",
        "Field Goal %",
        "3-Point Field Goals",
        "3-Point Field Goals Attempted",
        "3-Point Field Goal %",
        "2-Point Field Goals",
        "2-Point Field Goals Attempted",
        "2-Point Field Goal %",
        "Effective Field Goal %",
        "Free Throws",
        "Free Throws Attempted",
        "Free Throw %",
        "Offensive Rebounds",
        "Defensive Rebounds",
        "Total Rebounds",
        "Assists",
        "Steals",
        "Blocks",
        "Turnovers",
        "Personal Fouls",
        "Points",
    ]
)

for team in team_abbrs:
    # for each team, it contains data of their roster and their stats
    with open(f'data/teams/{team}.html', encoding="cp437", errors='ignore') as f:
        page = f.read()

        # initialize soup object to parse html data
        soup = BeautifulSoup(page, "html.parser")

        # find table with the per game stats of the players
        players_stats_table = soup.find('table', id="per_game")

        # convert table to pandas dataframe
        stats_df = pd.read_html(str(players_stats_table))[0]

        # drop rk, age, g, gs columns
        stats_df = stats_df.drop(["Rk", "Age", "G", "GS"], axis=1)

        # filter the data, replace 'nan' with 0 values
        stats_df = stats_df.fillna(0)

        stats_df = stats_df.values.tolist()

        # insert the stats rows for each player into the dataframe
        for stats_row in stats_df:
            players_stats_df.loc[players_stats_df.shape[0]] = stats_row



Directory "csv_files" cannot be created


In [301]:
# need to adjust column data types from objects to floats

# iterate through the column names, and change all column types to float64
for col in list(players_stats_df.keys()):
    if col != 'Name':
        players_stats_df = players_stats_df.astype({col: float})

In [302]:
# adjust indexing on current data frame
players_stats_df.index += 1

# save it to a csv file

# create directory
dir_name = 'csv_files'

try:
    os.makedirs(dir_name)
except OSError as error:
    print(f'Directory "{dir_name}" cannot be created')

players_stats_df.to_csv('csv_files/players_stats.csv')

Directory "csv_files" cannot be created


# 5. Obtain coach details and save to csv file

In [253]:
coach_links = []
team_name = []

# obtain the links for the coaches
for team in team_abbrs:
    # for each team, it contains link to data of their coach
    with open(f'data/teams/{team}.html', encoding="cp437", errors='ignore') as f:
        page = f.read()

        # initialize soup object to parse html data
        soup = BeautifulSoup(page, "html.parser")

        # find the link for the coach details
        info = soup.find(id='meta').find(
            'div', {'data-template': "Partials/Teams/Summary"})

        # obtain the href of the coach link tag
        coach_link_tag = [p for p in info.find_all(
            'p') if 'Coach' in str(p)][0]
        coach_link = str(coach_link_tag.find('a')['href'])

        # add it to list of coach links
        coach_links.append(coach_link)


In [263]:
coach_df = pd.DataFrame(
    columns=[
        "Name",
        "Date of Birth",
        "Team"
    ]
)

for team, url, team_name in zip(team_abbrs, coach_links, team_names):
    html_data = requests.get(base_url + url)

    # get the html content of the url
    html = html_data.text

    # initialize soup object to parse html data
    soup = BeautifulSoup(html, "html.parser")

    # get the name of the coach
    name = soup.find(id='meta').find('h1').text.strip()

    # dob
    dob = soup.find(id="necro-birth").text.strip()
    dob_obj = datetime.strptime(dob, "%B %d, %Y")
    dob = str(dob_obj).split(' ')[0]

    coach_df.loc[coach_df.shape[0]] = [name, dob, team_name]



Directory "csv_files" cannot be created


In [304]:
# change type of dob to datetime64
coach_df['Date of Birth'] = pd.to_datetime(coach_df['Date of Birth'])

In [305]:
# adjust indexing on current data frame
coach_df.index += 1

# save it to a csv file

# create directory
dir_name = 'csv_files'

try:
    os.makedirs(dir_name)
except OSError as error:
    print(f'Directory "{dir_name}" cannot be created')

coach_df.to_csv('csv_files/coaches.csv')

Directory "csv_files" cannot be created


In [289]:
# here to check the types of the fields in the dataframes
teams_df.dtypes

Name                object
Abbreviated Name    object
Arena               object
Wins                 int64
Losses               int64
W-L                 object
dtype: object

In [296]:
teams_stats_df.dtypes

Field Goals                      float64
Field Goals Attempted            float64
Field Goal %                     float64
3-Point Field Goals              float64
3-Point Field Goals Attempted    float64
3-Point Field Goal %             float64
2-Point Field Goals              float64
2-Point Field Goals Attempted    float64
2-Point Field Goal %             float64
Free Throws                      float64
Free Throws Attempted            float64
Free Throw %                     float64
Offensive Rebounds               float64
Defensive Rebounds               float64
Total Rebounds                   float64
Assists                          float64
Steals                           float64
Blocks                           float64
Turnovers                        float64
Personal Fouls                   float64
Points                           float64
Name                              object
dtype: object

In [300]:
players_df.dtypes

Name                     object
Date of Birth    datetime64[ns]
Position                 object
Team                     object
dtype: object

In [303]:
players_stats_df.dtypes

Name                              object
Minutes Played                   float64
Field Goals                      float64
Field Goals Attempted            float64
Field Goal %                     float64
3-Point Field Goals              float64
3-Point Field Goals Attempted    float64
3-Point Field Goal %             float64
2-Point Field Goals              float64
2-Point Field Goals Attempted    float64
2-Point Field Goal %             float64
Effective Field Goal %           float64
Free Throws                      float64
Free Throws Attempted            float64
Free Throw %                     float64
Offensive Rebounds               float64
Defensive Rebounds               float64
Total Rebounds                   float64
Assists                          float64
Steals                           float64
Blocks                           float64
Turnovers                        float64
Personal Fouls                   float64
Points                           float64
dtype: object

In [306]:
coach_df.dtypes

Name                     object
Date of Birth    datetime64[ns]
Team                     object
dtype: object

### All data collected and saved to csv files under 'csv_files'
- now write script to create database tables, read data from csv files and insert to database (file is 'nba_script.py')
- filter/clean any data that might need

In [308]:
import psycopg2

In [311]:
# need to convert pandas data types to database data types
# pandas data type: postgres data type
type_replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

In [330]:
# loop through every column in the data frame, and append the postgres matching data type
# i.e., 'Name varchar, Wins int' to be used for create table statement

# teams_df.dtypes.replace(type_replacements) replaces the pandas types with our dictionary type mappings
# column_names = ", ".join([c, t for c, t in zip()])

col_types_strings = []

for (c, t) in zip(teams_df.columns, teams_df.dtypes.replace(type_replacements)):
    col_types_strings.append(f'{c} {t}')
    
column_string = ", ".join(col_types_strings)

column_string


'Name varchar, Abbreviated Name varchar, Arena varchar, Wins int, Losses int, W-L varchar'